test

In [1]:
import io
# from io import BytesIO
import fitz
import requests
from Source import *

import PyPDF2

from bs4 import BeautifulSoup
import pandas as pd
import datetime 
from decimal import Decimal

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException

url_chrome = r"D:\Chromedriver\chromedriver.exe"
url_ff =  r"D:\geckodriver.exe" 

def down_content(url):
    response = requests.get(url)
    return response.content

def extract_pdf(pdfcontent, num_page):
    text = ''
    with io.BytesIO(pdfcontent) as pdf_stream:
        pdf_all = fitz.open(stream=pdf_stream, filetype = 'pdf')

        if num_page >= 0 and num_page < pdf_all.page_count:
            page = pdf_all.load_page(num_page)
            text = page.get_text("text")
            return text


def extract_offline_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        num_pages = pdf_reader.numPages
        text = ''
        for page_num in range(num_pages):
            page = pdf_reader.getPage(page_num)
            text += page.extractText()
        return text

def convert_to_float(value):
    return round(Decimal(float(value.replace('.', '.').replace(',', '.').replace('%', '').replace('*', '').replace('[','').replace(']',''))),3)

### Sacombank

In [2]:
ini_STB_url  = 'https://www.sacombank.com.vn/cong-cu/lai-suat.html'
pdf_STB_url ='https://www.sacombank.com.vn/content/dam/sacombank/files/cong-cu/lai-suat/tien-gui/khcn/SACOMBANK_LAISUATNIEMYETTAIQUAY_KHCN_VIE.pdf'

ini_HDB_url = 'https://hdbank.com.vn/vi/personal/cong-cu/interest-rate'
pdf_HDB_url = 'https://cdn.hdbank.com.vn/hdbank-file/document/20230112BIEULAISUATTIENGUIKHACHHANGCANHANKHCNWEBSITE_1701341142767.pdf?_gl=1*1xmopsn*_ga*Mjk0OTE1MjgyLjE3MDA4ODI0NTg.*_ga_L6TMDVXKWJ*MTcwMTY1NzkzNi4zLjAuMTcwMTY1Nzk0MC41Ni4wLjA.'

In [3]:
#Get driver
driver = webdriver.Firefox(service=Service(executable_path=url_ff))
driver.maximize_window()

# Fetch the HTML content using requests
response = requests.get(ini_STB_url).content

# Parse the HTML using Beautiful Soup
SCB_ini_pdf = BeautifulSoup(response, 'html.parser').\
    find('div', class_='div-download__lang--wrapper').\
    find('p')['data-href']

pdf_STB_url = f'https://www.sacombank.com.vn/{STB_ini_pdf}'

# Close the WebDriver
driver.quit()

In [31]:
icontent = extract_pdf(down_content(pdf_STB_url), 3)
ini_lines = icontent.split('\n')
lines = [line.strip() for line in ini_lines]
Update_Int = pd.DataFrame()

for segment_type in ('AF', 'MAF', 'UMASS'):
    Rate_row = pd.DataFrame()
    Rate_row['MMYYYY'] = [datetime.datetime.now().strftime("%Y-%m-%d")]
    Rate_row['Bank_name'] = ['Sacombank']
    Rate_row['Bank_code'] = ['STB']
    Rate_row['Segment'] = [segment_type]
    for ky_han_index in [1, 2, 3, 6, 12, 13, 18, 24, 36]:
        String = f'{ky_han_index} tháng'
        if String in lines:
            value = lines[lines.index(String) + 1]
            Rate_row[f'{ky_han_index} tháng'] = [value]
    
    Update_Int = pd.concat([Update_Int, Rate_row], ignore_index=True)
Update_Int

,MMYYYY,Bank_name,Bank_code,Segment,1 tháng,2 tháng,3 tháng,6 tháng,12 tháng,13 tháng,18 tháng,24 tháng,36 tháng
0,2023-12-14,Sacombank,STB,AF,3.60%,3.70%,3.80%,4.70%,5.00%,5.00%,5.10%,5.15%,5.20%
1,2023-12-14,Sacombank,STB,MAF,3.60%,3.70%,3.80%,4.70%,5.00%,5.00%,5.10%,5.15%,5.20%
2,2023-12-14,Sacombank,STB,UMASS,3.60%,3.70%,3.80%,4.70%,5.00%,5.00%,5.10%,5.15%,5.20%


HDBank

In [12]:
url_HDB = "https://hdbank.com.vn/vi/personal/cong-cu/interest-rate"

driver = webdriver.Firefox(service=Service(executable_path=url_ff))
driver.maximize_window()

#IF infinite page load 
try:
    driver.implicitly_wait(60)
    driver.get(url_HDB)
    button = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/main/div[2]/div/div/div/div[2]/div/div/div[1]/a')
    button.click()
except:
    button = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/main/div[2]/div/div/div/div[2]/div/div/div[1]/a')
    button.click()

source = driver.page_source
soup = BeautifulSoup(source, 'html')
driver.close()

In [ ]:
HDB_ini_pdf = soup.find('div', class_='interest-rate__content__list col-sm-12  ').find_all('a')['href']
url_HB_PDF = f'https://www.sacombank.com.vn/{HDB_ini_pdf}'
url_HB_PDF

In [ ]:
HDB_content = extract_pdf(down_content(url_HB_PDF), 3)
ini_lines = HDB_content.split('\n')
lines = [line.strip() for line in ini_lines]
Update_Int = pd.DataFrame()

for segment_type in ('AF', 'MAF', 'UMASS'):
    Rate_row = pd.DataFrame()
    Rate_row['MMYYYY'] = [datetime.datetime.now().strftime("%Y-%m-%d")]
    Rate_row['Bank_name'] = ['Sacombank']
    Rate_row['Bank_code'] = ['STB']
    Rate_row['Segment'] = [segment_type]
    for ky_han_index in [1, 2, 3, 6, 12, 13, 18, 24, 36]:
        String = f'{ky_han_index} tháng'
        if String in lines:
            value = lines[lines.index(String) + 1]
            Rate_row[f'{ky_han_index} tháng'] = [value]
    
    Update_Int = pd.concat([Update_Int, Rate_row], ignore_index=True)
Update_Int

Techcombank

In [5]:
# import os

# pdf_TCB_url = 'https://media.techcombank.com/uploads/PLS-ONLINE-KHAC-20231205-0d06287982.pdf?_gl=1*15oriwt*_ga*MjA4NDU1MzgxNi4xNjk1MDk0NzYx*_ga_E643CMJPVE*MTcwMTk1MTUzNS42LjAuMTcwMTk1MTUzNS42MC4wLjA.'

# # def download_pdf(url, save_path):
# #     response = requests.get(url)
# #     with open(save_path, 'wb') as file:
# #         file.write(response.content)
# def download_pdf(url):
#     firefox_options = Options()
#     firefox_options.set_preference("browser.download.folderList", 2)
#     firefox_options.set_preference("browser.download.dir", r"C:\Users\hiennpd3\Downloads\Crawler")
#     firefox_options.set_preference("browser.download.useDownloadDir", True)
#     firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")  # Example for PDF files

#     driver = webdriver.Firefox(options=firefox_options, executable_path=url_ff)
#     driver.get(url)

#     download_link = driver.find_element(By.XPATH, "//*[@id='download']")
#     download_link.click()

#     # Wait for the file to be downloaded
#     wait = WebDriverWait(driver, 10)
#     wait.until(EC.visibility_of_element_located((By.XPATH, "//*[@id='download']")))

# # Close the browser
# driver.quit()

# def delete_pdf(save_path):
#         if os.path.exists(save_path):
#             os.remove(save_path)
#         else:
#             print(f"File '{save_path}' does not exist.")

# download_pdf(pdf_TCB_url, save_location)


In [2]:
import PyPDF2

pdf_path  = r"C:\Users\hiennpd3\Downloads\Crawler\PLS-ONL-KHAC-20231211-5ae8d4e9a6.pdf"

# Open the PDF file in read-binary mode
with open(pdf_path, 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Get the number of pages in the PDF
    num_pages = len(pdf_reader.pages)

    # Read the content of each page
    for page_number in range(num_pages):
        # Get a specific page
        page = pdf_reader.pages[page_number]

        # Extract the text from the page
        page_text = page.extract_text()

        # Print the text of the page
        print(f"Page {page_number + 1}:")
        print(page_text)
        print('\n')

Page 1:
 1/ 7 
   
I. LÃI SUẤT TIỆT KIỆM/TIỀN GỬI CÓ KỲ HẠN VND
VND SAVINGS/TERM DEPOSIT INTEREST RATE
≥3B 1 - 3B <1B ≥3B 1 - 3B <1B ≥3B 1 - 3B <1B ≥3B 1 - 3B <1B
1M 3.40 3.40 3.40 3.35 3.35 3.35 3.30 3.30 3.30 3.25 3.25 3.25
2M 3.40 3.40 3.40 3.35 3.35 3.35 3.30 3.30 3.30 3.25 3.25 3.25
3M 3.60 3.60 3.60 3.55 3.55 3.55 3.50 3.50 3.50 3.45 3.45 3.45
4M 3.60 3.60 3.60 3.55 3.55 3.55 3.50 3.50 3.50 3.45 3.45 3.45
5M 3.60 3.60 3.60 3.55 3.55 3.55 3.50 3.50 3.50 3.45 3.45 3.45
6M 4.70 4.65 4.60 4.65 4.60 4.55 4.60 4.55 4.50 4.55 4.50 4.45
7M 4.70 4.65 4.60 4.65 4.60 4.55 4.60 4.55 4.50 4.55 4.50 4.45
8M 4.70 4.65 4.60 4.65 4.60 4.55 4.60 4.55 4.50 4.55 4.50 4.45
9M 4.75 4.70 4.65 4.70 4.65 4.60 4.65 4.60 4.55 4.60 4.55 4.50
10M 4.75 4.70 4.65 4.70 4.65 4.60 4.65 4.60 4.55 4.60 4.55 4.50
11M 4.75 4.70 4.65 4.70 4.65 4.60 4.65 4.60 4.55 4.60 4.55 4.50
12M 5.00 4.95 4.90 4.95 4.90 4.85 4.90 4.85 4.80 4.85 4.80 4.75
13M 5.00 4.95 4.90 4.95 4.90 4.85 4.90 4.85 4.80 4.85 4.80 4.75
14M 5.00 4.95 

In [22]:
import PyPDF2
import re
import pandas as pd

pdf_path = r"C:\Users\hiennpd3\Downloads\Crawler\PLS-ONL-KHAC-20231211-5ae8d4e9a6.pdf"

with open(pdf_path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    page_text = pdf_reader.pages[0].extract_text().split(' ')
    page_text_2 = [re.split(r'\n', word) for word in page_text if word != '\n']
    lines_2 = sum(page_text_2, [])

Rate_tb = pd.DataFrame()

for segment_no in range(3, 10, 3):
    Rate_row = pd.DataFrame()
    Rate_row['Segment'] = ['AF' if segment_no == 3 else 'MAF' if segment_no == 6 else 'UMASS']
    
    for ky_han_index in [1, 2, 3, 6, 12, 13, 18, 24, 36]:
        String = f'{ky_han_index}M'
        if String in lines_2:
            value = lines_2[lines_2.index(String) + segment_no]
            Rate_row[f'{ky_han_index} tháng'] = [value]
    
    Rate_tb = pd.concat([Rate_tb, Rate_row], ignore_index=True)

Rate_tb

,Segment,1 tháng,2 tháng,3 tháng,6 tháng,12 tháng,13 tháng,18 tháng,24 tháng,36 tháng
0,AF,3.40,3.40,3.60,4.60,4.90,4.90,4.90,4.90,4.90
1,MAF,3.35,3.35,3.55,4.55,4.85,4.85,4.85,4.85,4.85
2,UMASS,3.30,3.30,3.50,4.50,4.80,4.80,4.80,4.80,4.80
